In [1]:
%cd ../
%run embmi/main.py

In [ ]:
import matplotlib.pyplot as plt

In [2]:
for i in range(4):
    s = {}
    for k, v in data['exdata'][i].DIN:
        if k not in s.keys():
            s[k] = 0
        else:
            s[k] += 1

    print(f'session{i+1} : {sorted(s.items(), key=lambda x:x[0])}')

In [116]:
def adjustment(data):
    res = []
    for s in data:
        d = np.asarray([
            v[:, :] - (v[conf.experiment.Ref_ch[0], :] + v[conf.experiment.Ref_ch[1], :])/2
            for k, v in s.items()
        ])
        res.append(d)
    return res

target = ('master_true', 'master_fake', 'master_emg')
idx_coi = [i - 1 for i in conf.experiment.COI]

erp = {
    key: adjustment(data[key]) 
    for key in target
}

## Calc Mean by Session

In [88]:
erp_session = {
    k: [i.mean(0) for i in erp[k]]
    for k in target
}

In [152]:
df_session = [
    v
    for v in zip([erp_session[k] for k in target])
]

In [160]:
df_session[0]

([array([[ 13.10312537,  13.14341793,  13.18596084, ...,  44.17535643,
           44.05441362,  43.93181894],
         [-20.75937952, -20.68161721, -20.59474885, ..., -19.71114131,
          -19.79286232, -19.86867601],
         [-17.1819517 , -17.02539057, -16.86004992, ..., -12.08998721,
          -12.08028387, -12.06304861],
         ...,
         [ 51.86511008,  51.93370272,  52.00484728, ...,  52.42190458,
           52.50338685,  52.58733593],
         [-12.74864062, -12.56607328, -12.37976928, ..., -12.2750312 ,
          -12.19108155, -12.09759919],
         [ -8.87263226,  -8.78821098,  -8.6972236 , ...,  -3.65753998,
           -3.58338087,  -3.50357419]]),
  array([[  6.87980945,   6.99259521,   7.09734743, ...,  31.57631467,
           31.52099121,  31.46071546],
         [-29.87479646, -29.79770201, -29.72480139, ...,  -6.85077343,
           -6.85928893,  -6.87215276],
         [-24.05297567, -23.90602301, -23.77094024, ...,  -9.96658945,
           -9.97798812,  -9.99228

## Calc Mean by All Sessions

In [113]:
erp_all = {
    k: np.vstack(erp[k]).mean(0)
    for k in target
}

In [ ]:

df = [pd.DataFrame(e[idx_coi, :], columns=range(-1000, 1000), index=conf.experiment.COI) for e in erp]
df_mean = pd.DataFrame([d.mean(0) for d in df], index=['true', 'fake', 'emg']).T

In [ ]:
df_mean

In [ ]:
df_mean.plot(title='True Fake and EMG (0 = DIN6, 7, 8)', figsize=(16, 9))

In [ ]:
target_range = range(1275, 1375)
df_peak = df_mean.iloc[target_range].apply(
    lambda x: pd.Series(scipy.signal.find_peaks(-x)[0] + 1275 - 1)
)
df_mean.iloc[range(1275, 1375)].plot(figsize=(16, 9))
plt.vlines(df_peak-1000, -20, 0)
print(df_peak)

In [ ]:
# df_peak = df_peak.iloc[0:1, :].astype(int)
df_peak = pd.DataFrame({
    'true': [df_peak.true[0]],
    'fake': [df_peak.fake[0]],
    'emg': [df_peak.emg[0]]
}).astype(np.int_)

In [ ]:
_df_true = df_mean.true
_df_true.index = range(-df_peak.true[0], 2000 - df_peak.true[0])
_df_fake = df_mean.fake
_df_fake.index = range(-df_peak.fake[0], 2000 - df_peak.fake[0])
_df_emg = df_mean.emg
_df_emg.index = range(-df_peak.emg[0], 2000 - df_peak.emg[0])

df_fix = pd.DataFrame([
    _df_true,
    _df_fake,
    _df_emg
]).T

df_fix.plot(title='True Fake and EMG with COI Data (0 = Peak), day2-session3', figsize=(16, 9))

In [ ]:
df_all = [pd.DataFrame(e, columns=range(-1000, 1000)) for e in erp]
df_mean_all = pd.DataFrame([d.mean(0) for d in df_all], index=['true', 'fake', 'emg']).T

_df_true = df_mean_all.true
_df_true.index = range(-df_peak.true[0], 2000 - df_peak.true[0])
_df_fake = df_mean_all.fake
_df_fake.index = range(-df_peak.fake[0], 2000 - df_peak.fake[0])
_df_emg = df_mean_all.emg
_df_emg.index = range(-df_peak.emg[0], 2000 - df_peak.emg[0])

df_fix_all = pd.DataFrame([
    _df_true,
    _df_fake,
    _df_emg
]).T

df_fix_all.plot(title='True Fake and EMG with ALL Data (0 = Peak)', figsize=(16, 9))

In [ ]:
df_fix_all.loc[-49:50].plot(title='Intrest Time TOI (Peak around 98ms) (0 = Peak)', figsize=(16, 9))
plt.vlines(0, -10, 0)

In [ ]:
df_fix_all.loc[-74:75].plot(title='Intrest Time TOI (Peak around 148ms) (0 = Peak)', figsize=(16, 9))
plt.vlines(0, -10, 0)

In [ ]:
df_fix_all.loc[-300:301].plot(title='Intrest Time TOI (Peak around 600ms) (0 = Peak)  day2-session4', figsize=(16, 9))
plt.vlines(0, -10, 0)